In [ ]:
# Install dependencies
!pip install -q datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

In [3]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
# Load tiny Bengali subset
from datasets import load_dataset, DatasetDict, Audio

common_voice = DatasetDict()
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_17_0", "bn",
                                   split="train[:100]+validation[:20]")
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_17_0", "bn",
                                  split="test[:20]")

# Cast audio to 16kHz
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

# Initialize processor
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Bengali", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Bengali", task="transcribe")

# Initialize model
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.config.forced_decoder_ids = None
model.generation_config.language = "bengali"
model.generation_config.task = "transcribe"

# Preprocessing function
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

# Apply preprocessing with reduced processes
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

# Data collator
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

# Lightweight training arguments
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./test-whisper-bn",
    per_device_train_batch_size=4,  # Reduced from 16
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=10,
    max_steps=100,  # Just 100 steps for testing
    gradient_checkpointing=False,  # Disable to save memory
    fp16=False,  # Disable mixed precision
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,  # Reduced from 8
    predict_with_generate=True,
    generation_max_length=100,  # Reduced from 225
    save_steps=50,
    eval_steps=50,
    logging_steps=10,
    report_to=[],
    load_best_model_at_end=False,
    push_to_hub=False,  # Disable Hub push for testing
)

# Evaluation metric
import evaluate
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

# Initialize trainer
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

# Start test training
try:
    trainer.train()
except Exception as e:
    print(f"Training failed: {str(e)}")
    print("Consider reducing batch size further or using fewer samples")

# Cleanup
import torch
torch.cuda.empty_cache()

n_shards.json:   0%|          | 0.00/17.5k [00:00<?, ?B/s]

bn_train_0.tar:   0%|          | 0.00/747M [00:00<?, ?B/s]

bn_dev_0.tar:   0%|          | 0.00/359M [00:00<?, ?B/s]

bn_test_0.tar:   0%|          | 0.00/367M [00:00<?, ?B/s]

bn_other_2.tar:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

...

bn_other_23.tar:   0%|          | 0.00/602M [00:00<?, ?B/s]

bn_other_24.tar:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

bn_invalidated_0.tar:   0%|          | 0.00/296M [00:00<?, ?B/s]

bn_validated_0.tar:   0%|          | 0.00/1.49G [00:00<?, ?B/s]

bn_validated_1.tar:   0%|          | 0.00/138M [00:00<?, ?B/s]